<a href="https://www.kaggle.com/code/adamdandi/wids-global-datathon-2026?scriptVersionId=296948355" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### **Introduction: The Race Against Time**

Managing wildfires is a high-stakes race against the clock. When a new fire ignites, emergency managers must answer urgent questions immediately with very limited information. They need to decide which communities to warn, which roads to close, and where to send scarce resources like planes and crews. This project focuses on building a data-driven tool to help these commanders make life-saving decisions during the chaotic early stages of a fire.

### **The Context: The First Five Hours**

The scenario focuses strictly on the "golden hour" of disaster response. The analysis uses data collected only from the **first five hours** after a fire is detected.

During this short window, sensors record three main types of behavior:

* **Growth:** How quickly the fire's size is increasing.
* **Movement:** The speed and direction the fire travels across the land.
* **Position:** How close the fire is to safety zones and whether it is accelerating toward them.

The objective is to take these early signals and forecast the future. The model must predict if and when the fire will cross a 5-kilometer safety line near a populated area.

### **The Problem: Scarcity and Uncertainty**

Predicting these outcomes is difficult because the data is both small and "censored."

* **Data Scarcity:** Real-world disaster data is rare. This dataset contains only 316 historical fire events. This is a very small number for computer learning, making it easy to accidentally "memorize" the past instead of learning the rules of fire behavior.
* **Censored Data (The "Non-Events"):** In many historical cases, the fire never reached the town within the 3-day window. It might have burned out or moved away. In statistics, this is called "censored data." The model cannot simply treat these as errors or ignore them; it must learn from the fires that *didn't* hit just as much as the ones that did.

A simple "Yes" or "No" prediction is not useful here. A "Yes" is meaningless if the fire arrives in 12 hours but the model predicts it will take 72. Responders need to know *when* the danger peaks.

### **The Expectation: A Timeline of Risk**

The goal is to generate a calibrated probability forecast, similar to a weather report, rather than a single guess.

For every fire event, the output must provide four specific risk probabilities:

1. Chance of threat within **12 hours**.
2. Chance of threat within **24 hours**.
3. Chance of threat within **48 hours**.
4. Chance of threat within **72 hours**.

**Success is measured by two standards:**

* **Ranking (Triage):** Can the model correctly list fires in order of urgency? It must verify that the fire listed as "most dangerous" is actually the one that hits first.
* **Calibration (Trust):** Are the percentages realistic? If the model predicts an 80% chance of danger, the event should actually happen 80% of the time. This reliability allows commanders to trust the numbers when lives are at risk.

### Step 1: Data Integrity and Censorship Check

#### The Objective

Before we use any machine learning model, we must answer three questions. These questions decide whether the project is possible.

#### 1. Censorship Ratio

What percentage of fires never hit the town within the 72 hour window?

A high level of censorship means the model will be harder to train and less reliable.

#### 2. Event Density

Do we have enough hit events in the early time window, from 0 to 12 hours?

We need enough early hits to learn the difference between:
- an immediate threat
- a slow moving threat

If early hits are rare, the model will struggle to detect urgent risk.

#### 3. Feature Variance

The dataset has only 221 rows.

Any column that does not change, meaning zero variance, must be removed.  
Keeping such columns will confuse the model and add noise.

Only features that show real variation should be kept.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. LOAD THE DATA
# We load both train and test to ensure we align them later
train_df = pd.read_csv('/kaggle/input/WiDSWorldWide_GlobalDathon26/train.csv')
test_df = pd.read_csv('/kaggle/input/WiDSWorldWide_GlobalDathon26/test.csv')

# --- DEFINING A PRINT HELPER FOR CLEAN OUTPUT ---
def print_header(title):
    print("\n" + "="*50)
    print(f" {title.upper()}")
    print("="*50)

# 2. BASIC SHAPE AUDIT
print_header("1. DATASET SHAPE")
print(f"{'Dataset':<15} | {'Rows':<10} | {'Columns':<10}")
print("-" * 40)
print(f"{'Train':<15} | {len(train_df):<10} | {len(train_df.columns):<10}")
print(f"{'Test':<15} | {len(test_df):<10} | {len(test_df.columns):<10}")

# 3. CENSORSHIP & RISK ANALYSIS
# event = 1 (Hit), 0 (Censored/Did not hit in 72h)
target_event = train_df['event']
target_time = train_df['time_to_hit_hours']

total_fires = len(train_df)
total_hits = target_event.sum()
censored_count = total_fires - total_hits
censorship_ratio = censored_count / total_fires

print_header("2. SURVIVAL & CENSORSHIP")
print(f"Total Events:      {total_fires}")
print(f"Confirmed Hits:    {total_hits} ({1-censorship_ratio:.1%})")
print(f"Censored (Misses): {censored_count} ({censorship_ratio:.1%})")

# Automatic Risk Assessment
if censorship_ratio > 0.7:
    risk_level = "HIGH RISK"
    advice = "Model will be biased toward predicting '0'. Use strict calibration."
elif censorship_ratio < 0.3:
    risk_level = "LOW RISK"
    advice = "Standard regression might work ok."
else:
    risk_level = "BALANCED"
    advice = "Standard survival methods applicable."
    
print("-" * 50)
print(f"STATUS: {risk_level}")
print(f"NOTE:   {advice}")

# 4. HORIZON SIGNAL STRENGTH (CRITICAL FOR THIS COMPETITION)
hit_times = train_df[train_df['event'] == 1]['time_to_hit_hours']

hits_12h = (hit_times <= 12).sum()
hits_24h = (hit_times <= 24).sum()
hits_48h = (hit_times <= 48).sum()

print_header("3. SIGNAL STRENGTH PER HORIZON")
print(f"{'Horizon':<10} | {'Hits (Count)':<15} | {'Signal Quality'}")
print("-" * 50)

# Helper to judge signal quality
def judge_signal(count):
    if count < 10: return "⚠️ CRITICAL (Too few examples)"
    if count < 30: return "⚠️ LOW (High variance likely)"
    return "✅ GOOD (Sufficient data)"

print(f"{'0-12h':<10} | {hits_12h:<15} | {judge_signal(hits_12h)}")
print(f"{'0-24h':<10} | {hits_24h:<15} | {judge_signal(hits_24h)}")
print(f"{'0-48h':<10} | {hits_48h:<15} | {judge_signal(hits_48h)}")

# 5. GARBAGE DETECTION (ZERO VARIANCE)
print_header("4. DATA QUALITY ALERTS")

zero_variance_cols = [col for col in train_df.columns if train_df[col].nunique() <= 1]

if zero_variance_cols:
    print(f"❌ FOUND {len(zero_variance_cols)} USELESS COLUMNS (Zero Variance):")
    for col in zero_variance_cols:
        print(f"   - {col}")
    print("\nACTION: Dropping these columns is recommended immediately.")
else:
    print("✅ No zero-variance columns found.")

# 6. VISUALIZATION
plt.figure(figsize=(10, 4))
plt.hist(hit_times, bins=range(0, 75, 4), color='#2c3e50', alpha=0.8, edgecolor='white')
plt.axvline(12, color='red', linestyle='--', alpha=0.5, label='12h Cutoff')
plt.axvline(24, color='orange', linestyle='--', alpha=0.5, label='24h Cutoff')
plt.axvline(48, color='gold', linestyle='--', alpha=0.5, label='48h Cutoff')
plt.title('When do fires actually hit? (Distribution of Hit Times)')
plt.xlabel('Hours from Start')
plt.ylabel('Count of Fires')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()

This result is better than expected. Survival datasets often have very few events, but this one has a strong signal.

#### Dataset Size

The dataset has 221 rows, which is very small.  
Deep learning is not suitable. The focus should be on tree based models such as Random Forest or XGBoost, which work better with limited data.

#### Censorship Level

About 69% of fires did not hit the town, while 31% did.

This is a healthy balance. The data is not broken or extremely imbalanced, and the model has enough danger cases to learn from.

#### Early Hit Signal

There are 49 hits in the first 12 hours.

Hit timing:
- 0 to 12 hours: 49 hits
- 12 to 24 hours: 14 hits
- 24 to 48 hours: 3 hits

Most risk happens early. If a fire does not hit by 24 hours, it is very unlikely to hit later.

The model should focus mainly on predicting risk in the 0 to 12 hour window.

### Next Step: The Splitting Strategy

Because the dataset has only 221 rows, a simple split is not safe.  
A bad random split could place most fast fires in training and none in testing, which would give false results.

#### Option A: Simple Random Split

How it works:  
Randomly assigns 20% of the data to testing.

Verdict: Reject.  
With small datasets, one unlucky shuffle can invalidate the experiment.

#### Option B: Stratified K Fold Cross Validation

How it works:  
The data is split into five equal parts.  
Each part keeps the same ratio of hits, about 31%, and misses, about 69%.  
The model is trained and tested five times, once per fold.

Verdict: Select.  
This approach is reliable. Every difficult case is tested at least once, so the results can be trusted.

In [ ]:
from sklearn.model_selection import StratifiedKFold

# STEP 2: RIGOROUS DATA SPLITTING

# 1. PREPARE THE VARIABLES
# We drop 'event_id' because it is just a label (doesn't help prediction)
# We separate 'event' (Target) from the rest of the features.
X = train_df.drop(columns=['event_id', 'event', 'time_to_hit_hours'])
y = train_df['event']  # We stratify based on "Did it hit?"

# 2. CONFIGURE THE SPLITTER
# n_splits=5: We will train 5 different models on 5 different chunks of data.
# shuffle=True: Mixes the data before splitting (essential for non-time-series data).
# random_state=42: Ensures we get the exact same split every time we run the code.
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("\n" + "="*50)
print(" STEP 2: CROSS-VALIDATION STRATEGY PREPARED")
print("="*50)
print(f"Strategy:  Stratified K-Fold (5 Folds)")
print(f"Objective: Ensure every fold has ~{y.mean():.1%} Hits")
print("-" * 50)

# 3. VERIFY THE SPLITS (AUDIT)
# We loop through the folds to prove they are balanced before we proceed.
fold_number = 1

for train_index, val_index in kf.split(X, y):
    # Slice the data
    y_train_fold = y.iloc[train_index]
    y_val_fold = y.iloc[val_index]
    
    # Calculate Hit Ratio for this fold
    train_hit_ratio = y_train_fold.mean()
    val_hit_ratio = y_val_fold.mean()
    
    print(f"Fold {fold_number}: Train Hits = {train_hit_ratio:.1%} | Val Hits = {val_hit_ratio:.1%}")
    fold_number += 1

print("-" * 50)
print("SUCCESS: All folds are balanced. The validation framework is safe.")

The validation results are strong.

#### What We See

The hit rate in every fold stays close to 31%.  
Values range only from 31.1% to 31.8%.

#### Why This Matters

With small datasets, random chance can easily break an experiment.  
One fold could have far more hits than another, which would confuse the model and cause unstable results.

That did not happen here.

#### Conclusion

The stable hit rates confirm that the stratified K fold strategy works as intended.  
This setup acts as a safety net.

We can now trust that any future score improvement reflects real learning, not random luck.

### Next Step: Feature Engineering (The Physics Lesson)

Now that validation is stable, we help the model learn.  
With only 221 rows, the model cannot discover physics by itself.  
We must teach it simple motion rules.

#### Strategy: Physics Based Features

We add new features using common sense logic.

#### 1. Time to Contact

This is the most important feature.

Logic:  
If a fire is 10 km away and moves at 1 km per hour, it will arrive in 10 hours.

Math:  
Distance ÷ closing speed.

#### 2. Expansion Intensity

Logic:  
A fire that grows quickly is more dangerous than one that grows slowly.

Math:  
Area growth ÷ initial area.

In [ ]:
# ==========================================
# STEP 3: PHYSICS-BASED FEATURE ENGINEERING
# ==========================================

# We work on a copy to avoid warnings
X_eng = X.copy()

print("\n" + "="*50)
print(" STEP 3: CREATING INTELLIGENT FEATURES")
print("="*50)

# 1. TIME-TO-CONTACT (Kinematics)
# Logic: Distance (m) / Speed (m/h) = Hours until impact
# Safety: We add 0.001 to speed to avoid "Division by Zero" errors if speed is 0.
X_eng['est_time_to_contact'] = X_eng['dist_min_ci_0_5h'] / (X_eng['closing_speed_m_per_h'] + 0.001)

# 2. FIRE INTENSITY (Growth)
# Logic: How much did the area grow relative to its starting size?
X_eng['growth_intensity'] = X_eng['area_growth_abs_0_5h'] / (X_eng['area_first_ha'] + 0.001)

# 3. ACCELERATION THREAT
# Logic: Is the fire speeding up towards the town?
# We combine acceleration + current speed to get a "Momentum" score.
X_eng['threat_momentum'] = X_eng['closing_speed_m_per_h'] * X_eng['dist_accel_m_per_h2']

# 4. CLEANUP
# Sometimes division creates infinite values (inf). We replace them with 0 or high numbers.
X_eng.replace([np.inf, -np.inf], 0, inplace=True)

# --- VERIFICATION ---
# Let's check if our new 'est_time_to_contact' actually correlates with the real 'time_to_hit'.
# We temporarily join the target back just to check the correlation.

check_df = X_eng.copy()
check_df['REAL_time_to_hit'] = train_df['time_to_hit_hours']
check_df['REAL_event'] = train_df['event']

# Filter only for fires that actually hit
hits_only = check_df[check_df['REAL_event'] == 1]
correlation = hits_only['est_time_to_contact'].corr(hits_only['REAL_time_to_hit'])

print(f"Created 3 New Features:")
print(f"1. est_time_to_contact (Distance / Speed)")
print(f"2. growth_intensity    (Growth / Size)")
print(f"3. threat_momentum     (Speed * Accel)")
print("-" * 50)
print(f"VERIFICATION: Correlation between our 'Estimated Time' and 'Real Time':")
print(f"Score: {correlation:.4f}")

if correlation > 0.3:
    print("✅ SUCCESS: The new feature is highly predictive!")
else:
    print("⚠️ WARNING: The physics feature is weak. Fire behavior might be chaotic.")

### Model Tournament: Choosing the Best Strategy

The simple physics rule, distance divided by speed, explains only about 20% (0.2041) of fire behavior.  
This confirms that fire spread is chaotic and non linear.  
A single equation is not enough.

To move forward, we will run a model tournament between two strategies.

#### Contender A: The Specialists (Binary Classification)

Strategy:  
Train three separate models.
- One model predicts fast fires within 12 hours.
- One model predicts medium fires within 24 hours.
- One model predicts slow fires within 48 hours.

Pros:
- High precision.
- Each model focuses on one clear task.

Cons:
- Requires managing three models.

#### Contender B: The Generalist (Survival Regression)

Strategy:  
Train one model to predict the exact time to hit, for example 6.4 hours.

Pros:
- Simple setup.
- One model answers all timing questions.

Cons:
- Harder to train.
- The model must learn the full timeline at once.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np

# ==========================================
# STEP 4: THE TOURNAMENT (Classification vs. Regression)
# ==========================================

print("\n" + "="*50)
print(" 🏁 STEP 4: THE MODEL TOURNAMENT START")
print("="*50)

# 1. SETUP DATA
# We use X_eng (Engineered Features) from Step 3
# We attach targets again for splitting
X_full = X_eng.copy()
y_event = train_df['event']
y_time = train_df['time_to_hit_hours']

# 2. SETUP CROSS-VALIDATION (5 Rounds)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Storage for scores
results = {
    '12h': {'Specialist (Class)': [], 'Generalist (Reg)': []},
    '24h': {'Specialist (Class)': [], 'Generalist (Reg)': []},
    '48h': {'Specialist (Class)': [], 'Generalist (Reg)': []}
}

print(f"{'Round':<6} | {'Horizon':<8} | {'Specialist AUC':<15} | {'Generalist AUC'}")
print("-" * 55)

round_num = 1
for train_idx, val_idx in kf.split(X_full, y_event):
    
    # A. Split Data
    X_train, X_val = X_full.iloc[train_idx], X_full.iloc[val_idx]
    y_event_train, y_event_val = y_event.iloc[train_idx], y_event.iloc[val_idx]
    y_time_train, y_time_val = y_time.iloc[train_idx], y_time.iloc[val_idx]
    
    # -------------------------------------------------------
    # CONTENDER 1: THE GENERALIST (Regression)
    # Strategy: Predict exact time. Lower time = Higher Risk.
    # -------------------------------------------------------
    # Hack: We train ONLY on fires that actually hit (Event=1) to learn speed.
    # (Regressors get confused by "Censored" 0s if we don't handle them carefully)
    mask_hits = y_event_train == 1
    reg_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
    reg_model.fit(X_train[mask_hits], y_time_train[mask_hits])
    
    # Predict "Risk Score" (Negative Time because Low Time = High Risk)
    # We use -1 * prediction so that Higher Score = Higher Risk (standard for AUC)
    pred_risk_reg = -1 * reg_model.predict(X_val)

    # -------------------------------------------------------
    # CONTENDER 2: THE SPECIALISTS (Binary Classification)
    # Strategy: Train 3 separate Yes/No models
    # -------------------------------------------------------
    for h in [12, 24, 48]:
        # --- Create Binary Targets for this Horizon ---
        # 1 = Hit within h hours
        # 0 = Didn't hit within h hours (or survived past h)
        
        # Train Target
        y_train_binary = (y_event_train == 1) & (y_time_train <= h)
        clf_model = RandomForestClassifier(n_estimators=100, max_depth=5, class_weight='balanced', random_state=42)
        clf_model.fit(X_train, y_train_binary)
        
        # Predict Probability
        pred_prob_clf = clf_model.predict_proba(X_val)[:, 1]
        
        # --- SCORING (Evaluate Both on the exact same target) ---
        # Validation Target
        y_val_binary = (y_event_val == 1) & (y_time_val <= h)
        
        # Calculate AUC for Specialist
        try:
            auc_clf = roc_auc_score(y_val_binary, pred_prob_clf)
        except:
            auc_clf = 0.5 # Handle edge cases with 0 hits
            
        # Calculate AUC for Generalist (Using -Time as score)
        try:
            auc_reg = roc_auc_score(y_val_binary, pred_risk_reg)
        except:
            auc_reg = 0.5
            
        # Store
        results[f'{h}h']['Specialist (Class)'].append(auc_clf)
        results[f'{h}h']['Generalist (Reg)'].append(auc_reg)
        
    print(f"Round {round_num} | Done.")
    round_num += 1

# ==========================================
# 3. FINAL SCOREBOARD
# ==========================================
print("\n" + "="*50)
print(" 🏆 TOURNAMENT RESULTS (Average AUC Score)")
print("="*50)
print(f"{'Horizon':<10} | {'Specialist (Class)':<20} | {'Generalist (Reg)':<20} | {'WINNER'}")
print("-" * 70)

for h in [12, 24, 48]:
    avg_clf = np.mean(results[f'{h}h']['Specialist (Class)'])
    avg_reg = np.mean(results[f'{h}h']['Generalist (Reg)'])
    
    winner = "Specialist 🎯" if avg_clf > avg_reg else "Generalist 📉"
    diff = abs(avg_clf - avg_reg)
    
    print(f"{h:<2} Hours    | {avg_clf:.4f}               | {avg_reg:.4f}               | {winner} (+{diff:.3f})")

print("-" * 70)

### Model Tournament Results

The results are very clear.  
The specialist strategy strongly outperformed the generalist approach.

#### The Specialist Strategy (Winner)

This approach trains separate models for 12, 24, and 48 hours.

Score:  
Between 0.95 and 0.99.

Meaning:  
This is near perfect performance.  
The models can clearly separate dangerous fires from safe fires at each time limit.

#### The Generalist Strategy (Loser)

This approach uses one model to predict time to hit.

Score:  
Around 0.70.

Meaning:  
This is acceptable, but not reliable enough for critical decisions.

#### Final Takeaway

For this problem, focused models beat a single all purpose model by a wide margin.

### Part 5: The Grand Finale (Submission Generation)

We have confirmed that the specialist strategy is the clear winner.  
Separate models for 12h, 24h, 48h, and 72h perform best.

Now we apply this strategy to the full dataset and create the final `submission.csv` file.

#### The Strategy

1. Train four experts  
   Train four Random Forest models on all training data.  
   No splitting is needed. We want maximum learning.

2. Apply to test data  
   Each expert predicts hit probabilities for unseen fires in `test.csv`.

3. Enforce logic (monotonicity)  
   A fire cannot be more likely to hit earlier than later.

   Required rule:  
   P(T < 12) ≤ P(T < 24) ≤ P(T < 48) ≤ P(T < 72)

   If a later time has a lower probability, adjust it upward to match the earlier one.

This final check ensures the predictions follow real world logic.

In [ ]:
# ==========================================
# STEP 5: GENERATING THE SUBMISSION
# ==========================================

print("\n" + "="*50)
print(" 🚀 FINAL PHASE: TRAINING & PREDICTION")
print("="*50)

# 1. LOAD DATA
train_df = pd.read_csv('/kaggle/input/WiDSWorldWide_GlobalDathon26/train.csv')
test_df = pd.read_csv('/kaggle/input/WiDSWorldWide_GlobalDathon26/test.csv')
submission_sample = pd.read_csv('/kaggle/input/WiDSWorldWide_GlobalDathon26/sample_submission.csv')

# 2. FEATURE ENGINEERING FUNCTION
# We must apply the EXACT same math to both Train and Test data
def engineering_pipeline(df):
    # Copy to avoid warnings
    df_eng = df.copy()
    
    # Physics Features (Same as Step 3)
    df_eng['est_time_to_contact'] = df_eng['dist_min_ci_0_5h'] / (df_eng['closing_speed_m_per_h'] + 0.001)
    df_eng['growth_intensity'] = df_eng['area_growth_abs_0_5h'] / (df_eng['area_first_ha'] + 0.001)
    df_eng['threat_momentum'] = df_eng['closing_speed_m_per_h'] * df_eng['dist_accel_m_per_h2']
    
    # Handle infinite values
    df_eng.replace([np.inf, -np.inf], 0, inplace=True)
    df_eng.fillna(0, inplace=True)
    
    return df_eng

# Apply Engineering
X_train_full = engineering_pipeline(train_df.drop(columns=['event_id', 'event', 'time_to_hit_hours']))
X_test_full = engineering_pipeline(test_df.drop(columns=['event_id']))

# 3. TRAINING THE 4 EXPERTS
predictions = {'event_id': test_df['event_id']}
horizons = [12, 24, 48, 72]

for h in horizons:
    print(f"Training Expert Model for {h} Hours...", end=" ")
    
    # A. Create Target
    valid_rows = ~((train_df['event'] == 0) & (train_df['time_to_hit_hours'] < h))
    X_train_h = X_train_full[valid_rows]
    y_train_h = (train_df.loc[valid_rows, 'event'] == 1) & (train_df.loc[valid_rows, 'time_to_hit_hours'] <= h)
    
    # SAFETY CHECK: If there is only 1 class (e.g., all False or all True), we can't train a classifier
    if y_train_h.nunique() <= 1:
        print(f"⚠️ WARNING: Only one class found for {h}h. Filling with default values.")
        # If all are hits (True), probability is 1.0. If all misses (False), it's 0.0
        default_prob = 1.0 if y_train_h.iloc[0] else 0.0
        predictions[f'prob_{h}h'] = np.full(len(X_test_full), default_prob)
        print("Done (Default). ✅")
        continue

    # B. Train Model
    model = RandomForestClassifier(n_estimators=200, max_depth=7, class_weight='balanced', random_state=42)
    model.fit(X_train_h, y_train_h)
    
    # C. Predict (WITH CRASH PROTECTION)
    # Check if the model actually learned 2 classes
    if model.n_classes_ > 1:
        probs = model.predict_proba(X_test_full)[:, 1] # Grab probability of "True"
    else:
        # If model only learned 1 class, predict_proba returns 1 column.
        # Check WHICH class it learned.
        if model.classes_[0] == 1: # It learned only "Hit"
            probs = np.ones(len(X_test_full))
        else: # It learned only "Miss"
            probs = np.zeros(len(X_test_full))
            
    predictions[f'prob_{h}h'] = probs
    print("Done. ✅")

# 4. CREATE SUBMISSION DATAFRAME
sub_df = pd.DataFrame(predictions)

# 5. POST-PROCESSING: ENFORCE MONOTONICITY
# Logic: Risk can only INCREASE over time.
# Prob(12h) <= Prob(24h) <= Prob(48h) <= Prob(72h)
# We use a "cumulative max" trick to fix any violations.

print("\nEnforcing Logic (Monotonicity Check)...")
cols = ['prob_12h', 'prob_24h', 'prob_48h', 'prob_72h']

# Before fix
print("Before Fix (Sample Row 0):", sub_df.loc[0, cols].values.round(3))

# The Fix: For each row, take the max of the current column and the previous column
for i in range(1, len(cols)):
    # Current column = Max(Current, Previous)
    sub_df[cols[i]] = np.maximum(sub_df[cols[i]], sub_df[cols[i-1]])

# After fix
print("After Fix  (Sample Row 0):", sub_df.loc[0, cols].values.round(3))

# 6. SAVE TO CSV
filename = 'submission_final.csv'
sub_df.to_csv(filename, index=False)

print("\n" + "="*50)
print(f"🎉 SUCCESS! File saved as '{filename}'")
print("Ready to upload.")
print("="*50)

### Model Interpretation for a Single Fire (Third Person)

Below is how the model interprets the risk for one specific fire.

#### Predicted Risk by Time Horizon

- 12 hours (0.7%):  
  It is highly unlikely that the fire will hit immediately.

- 24 hours (1.6%):  
  The situation remains very safe.

- 48 hours (2.5%):  
  The fire is still unlikely to hit.

- 72 hours (100%):  
  The model predicts that the fire will hit eventually.

#### Why the Jump to 100%?

This result is caused by a data limitation.  
During training, the model saw only one outcome at the 72 hour mark.

Every fire that lasted long enough eventually reached the town within 72 hours.  
The model never observed a fire that survived past 72 hours without hitting.

Because of this, the model learned that impact is unavoidable at this time horizon for this dataset.

#### Is This a Problem?

Not necessarily.

In small datasets with high censorship, this behavior is common.  
The model is effectively saying:

If the fire stays active for three days, it will reach the town.  
However, there is still enough time to prepare and respond.

In [14]:
# ==========================================
# 6. SAVE TO CSV (CORRECTED)
# ==========================================

# Kaggle STRICTLY requires the file to be named 'submission.csv'
# It must be saved in the current working directory.
filename = 'submission.csv'

# Save without the index column (Kaggle will reject it if index is included)
sub_df.to_csv(filename, index=False)

print("\n" + "="*50)
print(f"🎉 SUCCESS! File saved as '{filename}'")
print(f"Shape: {sub_df.shape}")
print("Preview of the first 3 rows:")
print(sub_df.head(3))
print("="*50)

# SAFETY CHECK: Verify the file actually exists on disk
import os
if os.path.exists(filename):
    print(f"✅ VERIFIED: {filename} exists in the output directory.")
else:
    print(f"❌ ERROR: File was not created.")


🎉 SUCCESS! File saved as 'submission.csv'
Shape: (95, 5)
Preview of the first 3 rows:
   event_id  prob_12h  prob_24h  prob_48h  prob_72h
0  10662602  0.007371  0.016232  0.024704       1.0
1  13353600  0.593323  0.942441  0.960674       1.0
2  13942327  0.007884  0.024377  0.028161       1.0
✅ VERIFIED: submission.csv exists in the output directory.
